In [1]:
from requests import get, post, put
import os
import pandas as pd
from sensitive import *


In [119]:
auth = f"Bearer {AUTH_TOKEN}"
headers = {"Accept": "application/json",
           "Content-Type": "application/json", "Authorization": auth}


In [3]:
artists = {}
songs = []


def get_total_song_count():
    playlist_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)

    return list


def get_song_genres(song):
    genres = []
    for artist in song["track"]["artists"]:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(
                f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            try:
                artist_genres = artist_genres_res.json()["genres"]
            except Exception:
                print(artist_genres_res.json())
            genres = add_genres(genres, artist_genres)
            artists[artist["name"]] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres


def song_data(song):
    return song["track"]['id'], song["track"]['name'], song["track"]['popularity'], [artist["name"] for artist in song["track"]["artists"]]


In [4]:
total_song_count = get_total_song_count()

In [5]:


all_genres = []
for offset in range(0, total_song_count, 100):
    all_genres_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name, popularity, artist), song_genres = song_data(
            song), get_song_genres(song)
        songs.append({"id": id, "name": name, "artists": artist,
                     "popularity": popularity, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [6]:
songs = songs[-total_song_count:]

In [7]:
playlist = pd.DataFrame(data=songs, columns=[
                        'id', 'name', 'artists', 'popularity', 'genres'])


In [8]:
playlist


,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"[Justin Bieber, Dominic Fike]",57,"[canadian pop, pop, alternative pop rock]"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"[Kina, Snøw]",83,"[sad lo-fi, sad rap]"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,[Alessia Cara],77,"[canadian contemporary r&b, canadian pop, danc..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"[Teqkoi, Snøw]",69,"[lo-fi chill, sad lo-fi, sad rap]"
4,3xMTM729pJSwUv55WwWcm2,change,[One Hope],44,[sad lo-fi]
...,...,...,...,...,...
981,6OVpQb3zarqu0vfQav0Wwx,thenineteenseventyfive,[push baby],53,[uk pop]
982,0dOY7HTtl9OhlouW5ToZNC,"I'm Leaving, Sorry For Your Loss",[iamnotshane],50,[]
983,7byGwLkiIzJdHOZKZMS8nd,Something Super Sweet,[Rory Webley],65,"[alt z, social media pop]"
984,5ARrWiDDRDocvURbemcnCy,Cut My Fingers Off,[Ethan Bortnick],51,[]


In [9]:
playlist.to_csv('playlist.csv', header=True, index=None)

In [10]:
all_artists = list(artists.keys())
playlist = pd.read_csv('playlist.csv')
playlist

,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"['Justin Bieber', 'Dominic Fike']",57,"['canadian pop', 'pop', 'alternative pop rock']"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"['Kina', 'Snøw']",83,"['sad lo-fi', 'sad rap']"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,['Alessia Cara'],77,"['canadian contemporary r&b', 'canadian pop', ..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"['Teqkoi', 'Snøw']",69,"['lo-fi chill', 'sad lo-fi', 'sad rap']"
4,3xMTM729pJSwUv55WwWcm2,change,['One Hope'],44,['sad lo-fi']
...,...,...,...,...,...
981,6OVpQb3zarqu0vfQav0Wwx,thenineteenseventyfive,['push baby'],53,['uk pop']
982,0dOY7HTtl9OhlouW5ToZNC,"I'm Leaving, Sorry For Your Loss",['iamnotshane'],50,[]
983,7byGwLkiIzJdHOZKZMS8nd,Something Super Sweet,['Rory Webley'],65,"['alt z', 'social media pop']"
984,5ARrWiDDRDocvURbemcnCy,Cut My Fingers Off,['Ethan Bortnick'],51,[]


In [11]:
def genres_indexed(genres: list[str]) -> list[int]:
    indexed = []
    for all_genres_x in all_genres:

        continue_outer = False
        for genre in genres:
            index = all_genres_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


def artists_indexed(artists: list[str]) -> list[int]:
    indexed = []
    for all_artists_x in all_artists:

        continue_outer = False
        for genre in artists:
            index = all_artists_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


In [12]:
playlist["genres_indexed"] = [genres_indexed(
    eval(playlist["genres"][x])) for x in range(len(playlist["genres"]))]


In [13]:
playlist.columns = playlist.columns.map(lambda x: x.strip())

In [14]:
playlist["artists_indexed"] = [artists_indexed(
    eval(playlist["artists"][x])) for x in range(len(playlist["artists"]))]


In [15]:
playlist.columns


Index(['id', 'name', 'artists', 'popularity', 'genres', 'genres_indexed',
       'artists_indexed'],
      dtype='object')

## KNN


In [16]:
def knn_prepared_data(df):
    df = df[['id', 'name', 'genres', 'artists',
             'popularity', 'genres_indexed', 'artists_indexed']]
    list = []
    for index in range(len(df['id'])):
        list.append({'id': df['id'][index], 'name': df['name'][index], 'genres': df['genres'][index], 'artists': df['artists'][index],
                    'popularity': df['popularity'][index], 'genres_indexed': df['genres_indexed'][index], 'artists_indexed': df['artists_indexed'][index]})

    return list


In [17]:
song_dict = knn_prepared_data(playlist)

In [18]:
import operator


def getNeighbors(song, K):
    distances = []
    for song_index, song_value in enumerate(song_dict):
        if (song_index != song):
            dist = ComputeDistance(song_dict[song], song_value)
            distances.append((song_index, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors


In [19]:
def list_distance(a, b):
    distance = 0
    for item_a, item_b in list(zip(a, b)):
        distance += 0 if item_a == item_b else 0.2 if (len(a) < 400) else 0.1

    return distance

In [20]:
def ComputeDistance(a, b):
    genres_distance = list_distance(a['genres_indexed'], b['genres_indexed'])
    artists_distance = list_distance(a['artists_indexed'], b['artists_indexed'])
    popularity_distance = abs(a['popularity'] - b['popularity'])
    
    return genres_distance + artists_distance * 0.4 + popularity_distance * 0.003


## Song options

### Choose one of these to get the 50 closest songs to it


In [217]:
# Set the song name below
top_5 = get('https://api.spotify.com/v1/me/top/tracks?time_range=short_term&limit=5', headers=headers).json()
song_name = "Stay"
# top_5_names = map(lambda song: song['name'],top_5['items'])
# for i in top_5_names:
#     if i in playlist['name']:
#         song_name = i
#         break

In [218]:
K = 50
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
neighbors = getNeighbors(index, K)
print(song_dict[index]['name'], song_dict[index]['artists'],
      song_dict[index]['genres'], song_dict[index]['popularity'])
for neighbor, index in zip(neighbors, range(1, K + 1)):
    print(
        f'{index}. {song_dict[neighbor]["name"]} \t {song_dict[neighbor]["artists"]} \t {song_dict[neighbor]["genres"]} \t {song_dict[neighbor]["popularity"]}')


Stay ['Post Malone'] ['dfw rap', 'melodic rap', 'rap'] 76
1. Only Wanna Be With You - Pokémon 25 Version 	 ['Post Malone'] 	 ['dfw rap', 'melodic rap', 'rap'] 	 72
2. Rich & Sad 	 ['Post Malone'] 	 ['dfw rap', 'melodic rap', 'rap'] 	 71
3. I Fall Apart 	 ['Post Malone'] 	 ['dfw rap', 'melodic rap', 'rap'] 	 82
4. Deja Vu 	 ['Post Malone', 'Justin Bieber'] 	 ['dfw rap', 'melodic rap', 'rap', 'canadian pop', 'pop'] 	 69
5. 16 	 ['Baby Keem'] 	 ['hip hop', 'rap'] 	 76
6. ghost town (voice memo) 	 ['Chloe George'] 	 [] 	 71
7. In Luv With U 	 ['Finn'] 	 [] 	 68
8. In My Head 	 ['Peter Manos'] 	 [] 	 68
9. Robbery 	 ['Juice WRLD'] 	 ['chicago rap', 'melodic rap'] 	 85
10. Constellations 	 ['Jade LeMac'] 	 [] 	 66
11. 3:15 (Breathe) 	 ['Russ'] 	 ['hawaiian hip hop', 'rap'] 	 65
12. Fairytale 	 ['Livingston'] 	 [] 	 65
13. Cinema - Acoustic 	 ['Gary Go'] 	 [] 	 63
14. Utah Freestyle 	 ['Russ'] 	 ['hawaiian hip hop', 'rap'] 	 63
15. aftermath 	 ['vaultboy'] 	 [] 	 62
16. Crazy in Love 	 ['Eden

In [213]:
import json

data = {"name": f"{song_name!r} Related", "description": f"Songs related to {song_name!r}", "public": False}
playlist_creation = post(f'https://api.spotify.com/v1/users/{USER_ID}/playlists', headers=headers, data=json.dumps(data))
new_id = playlist_creation.json()['id']

In [214]:
item = playlist[[playlist['name'][x] == song_name for x in range(len(playlist['name']))]]
index = item.index[0]
song_uris = f'spotify:track:{song_dict[index]["id"]}'
for neighbor in neighbors:
    song_uris += f',spotify:track:{song_dict[neighbor]["id"]}'

add_songs_req = post(f'https://api.spotify.com/v1/playlists/{new_id}/tracks?uris={song_uris}', headers=headers, data=json.dumps({}))
add_songs_req.json()

{'snapshot_id': 'MywwOTY0NWY1OTQ5YzU2NTNhMTNkYjQ5ZWRmM2RjYzA2Y2IwNDhjY2Zm'}